In [1]:
import torch
import numpy as np
import micca_model
import simulate_model_gaussian
torch.set_printoptions(linewidth = 200)

In [2]:
def sqrtm(X):
    U, l, VT = torch.linalg.svd(X)
    return((U * torch.sqrt(l)) @ VT)

In [3]:
n = 1000
p = [3, 3]
psum = np.concatenate([[0], np.cumsum(p, 0)]) 
d = 2
k = None

In [4]:
model = simulate_model_gaussian.generate_model(p, k, d)
data = model.simulate(n)
Y_all = torch.cat(data.Y, dim = 1)
Y_all = Y_all - torch.mean(Y_all, 0)
Sigma_tilde = (Y_all.T @ Y_all) / (n - 1)
W = torch.cat(model.W, dim = 1)
Phi = torch.block_diag(*model.Phi)

In [8]:
Y = [Y_m - torch.mean(Y_m, 0) for Y_m in data.Y]
S11 = (Y[0].T @ Y[0])/(n-1)
S22 = (Y[1].T @ Y[1])/(n-1)
S12 = (Y[0].T @ Y[1])/(n-1)
S11_inv2 = sqrtm(torch.linalg.pinv(S11, rcond = 1e-06))
S22_inv2 = sqrtm(torch.linalg.pinv(S22, rcond = 1e-06))
C12_tilde = S11_inv2 @ S12 @ S22_inv2

In [22]:
# Fit 2-D once, then 1-D twice with classic CCA
F1, F2, rho, V1, V2 = micca_model.cca(Y[0], Y[1], d)
print(rho)

f1_1, f2_1, rho_1, a1_1, a2_1 = micca_model.cca(Y[0], Y[1], 1)
print(rho_1)

P1 = torch.eye(3) - f1_1 @ f1_1.T @ S11
P2 = torch.eye(3) - f2_1 @ f2_1.T @ S22
Y_mod = [Y[0] @ P1, Y[1] @ P2]
Y_mod = [Y - torch.mean(Y, 0) for Y in Y_mod]

f1_2, f2_2, rho_2, a1_2, a2_2 = micca_model.cca(Y_mod[0], Y_mod[1], 1, center = True)
print(rho_2)

tensor([0.4583, 0.2244])
tensor([0.4583])
tensor([0.2244])


In [7]:
# Now once again, with ML solution implementation
# Fit 2-d once
W_hat, Phi_hat = micca_model.find_ML_params(Y, d = 2)
W1 = W_hat[:, psum[0]:psum[1]]
W2 = W_hat[:, psum[1]:psum[2]]

print(torch.linalg.svd(S11_inv2 @ W1.T @ W2 @ S22_inv2).S[psum[0]:psum[1]])

# Fit 1-d one time
W_hat, Phi_hat = micca_model.find_ML_params(Y, d = 1)
W1 = W_hat[:, psum[0]:psum[1]]
W2 = W_hat[:, psum[1]:psum[2]]
print(torch.linalg.svd(S11_inv2 @ W1.T @ W2 @ S22_inv2).S[psum[0]:psum[1]])

# Project out learned space and fit again
W1_tilde = W1 @ S11_inv2
W2_tilde = W2 @ S22_inv2
a1 = (W1_tilde / torch.sqrt(torch.sum(W1_tilde**2))).T
a2 = (W2_tilde / torch.sqrt(torch.sum(W2_tilde**2))).T
f1 = S11_inv2 @ a1
f2 = S22_inv2 @ a2
P1 = torch.eye(3) - f1 @ f1.T @ S11
P2 = torch.eye(3) - f2 @ f2.T @ S22

Y_mod = [Y[0] @ P1, Y[1] @ P2]
Y_mod = [Y - torch.mean(Y, 0) for Y in Y_mod]
S11_mod = (Y_mod[0].T @ Y_mod[0])/(n-1)
S22_mod = (Y_mod[1].T @ Y_mod[1])/(n-1)
S11_inv2_mod = sqrtm(torch.linalg.pinv(S11_mod, rcond = 1e-06))
S22_inv2_mod = sqrtm(torch.linalg.pinv(S22_mod, rcond = 1e-06))

W_hat, Phi_hat = micca_model.find_ML_params(Y_mod, d = 1)
W1 = W_hat[:, psum[0]:psum[1]]
W2 = W_hat[:, psum[1]:psum[2]]
print(torch.linalg.svd(S11_inv2_mod @ W1.T @ W2 @ S22_inv2_mod).S[psum[0]:psum[1]])

tensor([6.7842e-01, 5.8495e-01, 9.1917e-09])
tensor([6.7842e-01, 2.0567e-08, 1.4283e-08])
tensor([5.8495e-01, 2.2308e-08, 3.1802e-09])


In [8]:
# One more time with EM

# Fit 2-D once
std_normal = torch.distributions.Normal(0, 1)
W_curr = std_normal.sample([2, sum(p)])
Phi_curr = torch.eye(sum(p))
for i in range(2500):
    W_next, Phi_next = micca_model.EM_step_stable(W_curr, Phi_curr, Y_all, Sigma_tilde, p = p)
    delta_W = torch.sum((W_curr - W_next)**2)
    delta_Phi = torch.sum((Phi_curr - Phi_next)**2)
    W_curr = W_next
    Phi_curr = Phi_next
    if (i%100 == 0): print(delta_W, delta_Phi)
W_hat = W_curr
Phi_hat = Phi_curr
W1 = W_hat[:, psum[0]:psum[1]]
W2 = W_hat[:, psum[1]:psum[2]]
print(torch.linalg.svd(S11_inv2 @ W1.T @ W2 @ S22_inv2).S[psum[0]:psum[1]])

# Fit 1-D once
std_normal = torch.distributions.Normal(0, 1)
W_curr = std_normal.sample([1, sum(p)])
Phi_curr = torch.eye(sum(p))
for i in range(2500):
    W_next, Phi_next = micca_model.EM_step_stable(W_curr, Phi_curr, Y_all, Sigma_tilde, p = p)
    delta_W = torch.sum((W_curr - W_next)**2)
    delta_Phi = torch.sum((Phi_curr - Phi_next)**2)
    W_curr = W_next
    Phi_curr = Phi_next
    if (i%100 == 0): print(delta_W, delta_Phi)
W_hat = W_curr
Phi_hat = Phi_curr
W1 = W_hat[:, psum[0]:psum[1]]
W2 = W_hat[:, psum[1]:psum[2]]
print(torch.linalg.svd(S11_inv2 @ W1.T @ W2 @ S22_inv2).S[psum[0]:psum[1]])

# Project out learned space and fit again
W1_tilde = W1 @ S11_inv2
W2_tilde = W2 @ S22_inv2
a1 = (W1_tilde / torch.sqrt(torch.sum(W1_tilde**2))).T
a2 = (W2_tilde / torch.sqrt(torch.sum(W2_tilde**2))).T
f1_1 = S11_inv2 @ a1
f2_1 = S22_inv2 @ a2
P1 = torch.eye(3) - f1_1 @ f1_1.T @ S11
P2 = torch.eye(3) - f2_1 @ f2_1.T @ S22
Y_mod = [Y[0] @ P1, Y[1] @ P2]
Y_mod = [Y - torch.mean(Y, 0) for Y in Y_mod]
Y_mod_all = torch.cat(Y_mod, dim = 1)
S11_mod = (Y_mod[0].T @ Y_mod[0])/(n-1)
S12_mod = (Y_mod[0].T @ Y_mod[1])/(n-1)
S22_mod = (Y_mod[1].T @ Y_mod[1])/(n-1)
S11_inv2_mod = sqrtm(torch.linalg.pinv(S11_mod, rcond = 1e-06))
S22_inv2_mod = sqrtm(torch.linalg.pinv(S22_mod, rcond = 1e-06))
Sigma_tilde_mod = torch.cat([torch.cat([S11_mod, S12_mod], 1), torch.cat([S12_mod.T, S22_mod], 1)], 0)

std_normal = torch.distributions.Normal(0, 1)
W_curr = std_normal.sample([1, sum(p)])
Phi_curr = torch.eye(sum(p))
for i in range(2500):
    W_next, Phi_next = micca_model.EM_step_stable(W_curr, Phi_curr, Y_mod_all, Sigma_tilde_mod, p = p)
    delta_W = torch.sum((W_curr - W_next)**2)
    delta_Phi = torch.sum((Phi_curr - Phi_next)**2)
    W_curr = W_next
    Phi_curr = Phi_next
    if (i%100 == 0): print(delta_W, delta_Phi)
W_hat = W_curr
Phi_hat = Phi_curr
W1 = W_hat[:, psum[0]:psum[1]]
W2 = W_hat[:, psum[1]:psum[2]]
W1_tilde = W1 @ S11_inv2_mod
W2_tilde = W2 @ S22_inv2_mod
a1 = (W1_tilde / torch.sqrt(torch.sum(W1_tilde**2))).T
a2 = (W2_tilde / torch.sqrt(torch.sum(W2_tilde**2))).T
f1_2 = S11_inv2_mod @ a1
f2_2 = S22_inv2_mod @ a2
print(torch.linalg.svd(S11_inv2_mod @ W1.T @ W2 @ S22_inv2_mod).S[psum[0]:psum[1]])

tensor(16.6310) tensor(7.8043)
tensor(8.1300e-09) tensor(9.2266e-08)
tensor(7.4895e-09) tensor(8.4254e-08)
tensor(6.7699e-09) tensor(7.5840e-08)
tensor(6.2095e-09) tensor(6.8909e-08)
tensor(5.6104e-09) tensor(6.1765e-08)
tensor(5.1432e-09) tensor(5.6644e-08)
tensor(4.6462e-09) tensor(5.0534e-08)
tensor(4.2337e-09) tensor(4.5463e-08)
tensor(3.8241e-09) tensor(4.0933e-08)
tensor(3.5010e-09) tensor(3.7483e-08)
tensor(3.1439e-09) tensor(3.3311e-08)
tensor(2.8554e-09) tensor(2.9791e-08)
tensor(2.5526e-09) tensor(2.6765e-08)
tensor(2.3080e-09) tensor(2.3909e-08)
tensor(2.1134e-09) tensor(2.1839e-08)
tensor(1.9130e-09) tensor(1.9374e-08)
tensor(1.6929e-09) tensor(1.7092e-08)
tensor(1.5341e-09) tensor(1.5416e-08)
tensor(1.3659e-09) tensor(1.3462e-08)
tensor(1.2380e-09) tensor(1.2171e-08)
tensor(1.1179e-09) tensor(1.0958e-08)
tensor(1.0108e-09) tensor(9.8219e-09)
tensor(8.9793e-10) tensor(8.6321e-09)
tensor(8.1482e-10) tensor(7.6177e-09)
tensor([6.7826e-01, 5.8480e-01, 3.0529e-08])
tensor(1.666

In [9]:
print(f1_2.T @ S11 @ f1_2)
print(f2_2.T @ S22 @ f2_2)
print(f1_2.T @ S11_mod @ f1_2)
print(f2_2.T @ S22_mod @ f2_2)

tensor([[1.0781]])
tensor([[1.2029]])
tensor([[1.]])
tensor([[1.0000]])


In [110]:
F1

tensor([[ 0.1082,  0.4640],
        [-0.3541,  0.0287],
        [ 0.0775,  0.4144]])

In [112]:
f1_2

tensor([[0.4110],
        [0.2049],
        [0.3765]])

In [6]:
# Testing the conjecture with two datasets

# Fit model with EM
std_normal = torch.distributions.Normal(0, 1)
W_curr = std_normal.sample([d, sum(p)])
Phi_curr = torch.eye(sum(p))
for i in range(5000):
    # W_next, Phi_next = micca_model.EM_step_stable(W_curr, Phi_curr, Y_all, Sigma_tilde, p = p)
    W_next, Phi_next = micca_model.EM_step(W_curr, Phi_curr, Sigma_tilde, p = p)
    delta_W = torch.sum((W_curr - W_next)**2)
    delta_Phi = torch.sum((Phi_curr - Phi_next)**2)
    W_curr = W_next
    Phi_curr = Phi_next
    if (i%100 == 0): print(delta_W, delta_Phi, micca_model.loglik(W_curr.T @ W_curr + Phi_curr, Sigma_tilde, n))
W_em = W_curr
Phi_em = Phi_curr
print(micca_model.loglik(W_em.T @ W_em + Phi_em, Sigma_tilde, n))

W_ml, Phi_ml = micca_model.find_ML_params(Y, d = 2)
print(micca_model.loglik(W_ml.T @ W_ml + Phi_ml, Sigma_tilde, n))

tensor(3.1860) tensor(55.0333) tensor(12085.7266)
tensor(1.4777e-07) tensor(6.0834e-07) tensor(11336.6396)
tensor(1.7205e-11) tensor(6.2288e-11) tensor(11336.6367)
tensor(5.4073e-13) tensor(7.8693e-13) tensor(11336.6367)
tensor(1.1653e-12) tensor(7.8906e-12) tensor(11336.6367)
tensor(2.2307e-12) tensor(4.7669e-12) tensor(11336.6357)
tensor(7.1410e-13) tensor(9.8694e-12) tensor(11336.6367)
tensor(1.4924e-12) tensor(1.9655e-12) tensor(11336.6377)
tensor(8.4133e-13) tensor(1.3185e-11) tensor(11336.6377)
tensor(1.1042e-12) tensor(9.0008e-12) tensor(11336.6367)
tensor(8.0475e-13) tensor(1.4913e-11) tensor(11336.6367)
tensor(2.4394e-12) tensor(4.2011e-12) tensor(11336.6377)
tensor(2.6810e-12) tensor(4.8077e-12) tensor(11336.6367)
tensor(1.2372e-12) tensor(3.1566e-12) tensor(11336.6377)
tensor(6.4287e-13) tensor(4.0128e-12) tensor(11336.6377)
tensor(1.4860e-12) tensor(1.1475e-12) tensor(11336.6377)
tensor(7.6300e-13) tensor(9.9520e-12) tensor(11336.6367)
tensor(1.0927e-12) tensor(7.2564e-13) 

In [7]:
# The original conjecture in terms of SVD of W_tilde
W1 = W_em[:, psum[0]:psum[1]]
W2 = W_em[:, psum[1]:psum[2]]
W1_tilde = W1 @ S11_inv2
W2_tilde = W2 @ S22_inv2

_, S1, A1T = torch.linalg.svd(W1_tilde, full_matrices=False)
_, S2, A2T = torch.linalg.svd(W2_tilde, full_matrices=False)
F1 = S11_inv2 @ A1T.T
F2 = S22_inv2 @ A2T.T

model_cov_1 = torch.cat([torch.cat([S11, W1.T @ W2], axis = 1),
                         torch.cat([W2.T @ W1, S22], axis = 1)], axis = 0)
model_cov_2 = torch.cat([torch.cat([torch.eye(p[0]), W1_tilde.T @ W2_tilde], axis = 1),
                         torch.cat([W2_tilde.T @ W1_tilde, torch.eye(p[1])], axis = 1)], axis = 0)
gen_var = torch.cat([torch.cat([F1.T @ S11 @ F1, F1.T @ S12 @ F2], axis = 1),
                         torch.cat([(F1.T @ S12 @ F2).T, F2.T @ S22 @ F2], axis = 1)], axis = 0)
gen_var_2 = torch.cat([torch.cat([torch.eye(p[0]), A1T.T @ F1.T @ S12 @ F2 @ A2T], axis = 1),
                         torch.cat([(A1T.T @ F1.T @ S12 @ F2 @ A2T).T, torch.eye(p[1])], axis = 1)], axis = 0)
D_half = torch.block_diag(sqrtm(S11), sqrtm(S22))

Sigma_alt = D_half @ gen_var_2 @ D_half
print(micca_model.loglik(model_cov_1, Sigma_tilde, n))
print(micca_model.loglik(Sigma_alt, Sigma_tilde, n))
print(micca_model.loglik(W_ml.T @ W_ml + Phi_ml, Sigma_tilde, n))

tensor(11336.6367)
tensor(11336.6377)
tensor(11336.6367)


In [87]:
mat = F1.T @ S12 @ F2
Um, Sm, VmT = torch.linalg.svd(mat)
M1 = Um * torch.sqrt(Sm)
M2 = VmT.T * torch.sqrt(Sm)
W1_test =  S11 @ F1 @ M1
W2_test =  S22 @ F2 @ M2
Phi1_test = S11 - W1_test @ W1_test.T
Phi2_test = S22 - W2_test @ W2_test.T
W_test = torch.cat([W1_test, W2_test], axis = 0)
Phi_test = torch.block_diag(Phi1_test, Phi2_test)
Sigma_test = W_test @ W_test.T + Phi_test
print(micca_model.loglik(Sigma_test, Sigma_tilde, n))

tensor(10840.6182)


In [82]:
W_curr = W_test.T
Phi_curr = Phi_test.T
for i in range(10):
    # W_next, Phi_next = micca_model.EM_step_stable(W_curr, Phi_curr, Y_all, Sigma_tilde, p = p)
    W_next, Phi_next = micca_model.EM_step(W_curr, Phi_curr, Sigma_tilde, p = p)
    delta_W = torch.sum((W_curr - W_next)**2)
    delta_Phi = torch.sum((Phi_curr - Phi_next)**2)
    W_curr = W_next
    Phi_curr = Phi_next
    print(delta_W, delta_Phi, micca_model.loglik(W_curr.T @ W_curr + Phi_curr, Sigma_tilde, n))

tensor(2.4565e-13) tensor(3.4497e-12) tensor(10840.6191)
tensor(2.8880e-14) tensor(9.7167e-13) tensor(10840.6191)
tensor(1.4211e-13) tensor(5.4468e-13) tensor(10840.6191)
tensor(4.5519e-14) tensor(3.9924e-13) tensor(10840.6191)
tensor(1.0981e-13) tensor(4.5031e-13) tensor(10840.6182)
tensor(6.9778e-14) tensor(2.7467e-13) tensor(10840.6191)
tensor(3.4098e-14) tensor(6.4415e-13) tensor(10840.6191)
tensor(8.5501e-14) tensor(5.5156e-13) tensor(10840.6182)
tensor(1.3368e-13) tensor(4.9849e-13) tensor(10840.6191)
tensor(7.9992e-14) tensor(2.8266e-13) tensor(10840.6191)


In [9]:
(C12_tilde - W1_tilde.T @ W2_tilde) @ W2_tilde.T @ torch.linalg.inv(1 - W2_tilde @ W2_tilde.T)

tensor([[-1.0368e-06,  2.7544e-07],
        [ 7.3637e-07,  7.1142e-08],
        [ 4.8321e-07,  1.6124e-07]])

In [54]:
W1_ml = W_ml[:, psum[0]:psum[1]]
W2_ml = W_ml[:, psum[1]:psum[2]]
W1_ml_tilde = W1_ml @ S11_inv2
W2_ml_tilde = W2_ml @ S22_inv2
W1_ml_tilde.T @ W2_ml_tilde
A1T @ W1_ml_tilde.T @ W2_ml_tilde @ A2T.T

tensor([[ 0.4559, -0.0409],
        [ 0.0403,  0.2220]])

In [66]:

M1 @ M2.T

tensor([[ 0.4559, -0.0409],
        [ 0.0403,  0.2220]])

In [19]:
# Re-write in terms of a sequence
# Currently only d = 2
# a1 = (W1_tilde.T/torch.sqrt(torch.sum(W1_tilde**2, 1)))
# a2 = (W2_tilde.T/torch.sqrt(torch.sum(W2_tilde**2, 1)))
# f1 = S11_inv2 @ a1
# f2 = S22_inv2 @ a2

# f1_1 = f1[:, 0:1]
# f2_1 = f2[:, 0:1]
# f1_2 = f1[:, 1:2]
# f2_2 = f2[:, 1:2]

f1_1 = F1[:, 0:1]
f2_1 = F2[:, 0:1]
f1_2 = F1[:, 1:2]
f2_2 = F2[:, 1:2]

P1 = torch.eye(p[0]) - f1_1 @ f1_1.T @ S11
P2 = torch.eye(p[1]) - f2_1 @ f2_1.T @ S22
Y_mod = [Y[0] @ P1, Y[1] @ P2]
Y_mod = [Y - torch.mean(Y, 0) for Y in Y_mod]
Y_mod_all = torch.cat(Y_mod, dim = 1)
S11_mod = (Y_mod[0].T @ Y_mod[0])/(n-1)
S12_mod = (Y_mod[0].T @ Y_mod[1])/(n-1)
S22_mod = (Y_mod[1].T @ Y_mod[1])/(n-1)

Phi_1 = torch.tensor([[f1_1.T @ S11 @ f1_1, f1_1.T @ S12 @ f2_1],
                      [f1_1.T @ S12 @ f2_1, f2_1.T @ S22 @ f2_1]])
Phi_2 = torch.tensor([[f1_2.T @ S11 @ f1_2, f1_2.T @ S12 @ f2_2],
                      [f1_2.T @ S12 @ f2_2, f2_2.T @ S22 @ f2_2]])
print(torch.logdet(Phi_1) + torch.logdet(Phi_2))

tensor(-0.5830)


In [12]:
f1_2

tensor([], size=(10, 0))

In [16]:
Phi_1 = torch.tensor([[f1_1.T @ S11 @ f1_1, f1_1.T @ S12 @ f2_1],
                      [f1_1.T @ S12 @ f2_1, f2_1.T @ S22 @ f2_1]])
Phi_2 = torch.tensor([[f1_2.T @ S11_mod @ f1_2, f1_2.T @ S12_mod @ f2_2],
                      [f1_2.T @ S12_mod @ f2_2, f2_2.T @ S22_mod @ f2_2]])
print(torch.linalg.det(Phi_1)*torch.linalg.det(Phi_2))

tensor(0.3551)


In [14]:
f1_2.T @ S11_mod @ f1_2

tensor([[1.]])

In [26]:
# Stationary points of likelihood
print((Sigma_inv - Sigma_inv @ Sigma_tilde @ Sigma_inv)[psum[0]:psum[1], psum[0]:psum[1]])
print((Sigma_inv - Sigma_inv @ Sigma_tilde @ Sigma_inv)[psum[0]:psum[1], psum[1]:psum[2]])
print((Sigma_inv - Sigma_inv @ Sigma_tilde @ Sigma_inv)[psum[1]:psum[2], psum[1]:psum[2]])
print((Sigma_inv - Sigma_inv @ Sigma_tilde @ Sigma_inv)@ W_hat)

tensor([[-2.9802e-08,  2.9802e-08, -1.3784e-07],
        [-1.4901e-08,  8.9407e-08,  5.4017e-08],
        [-1.5460e-07,  5.4017e-08, -5.3644e-07]])
tensor([[-0.0012,  0.0009,  0.0008],
        [-0.0024,  0.0017,  0.0015],
        [ 0.0018, -0.0013, -0.0012]])
tensor([[-2.9802e-08, -6.7754e-08, -8.1956e-08],
        [-7.3225e-08, -1.4901e-07, -3.3528e-08],
        [-6.7055e-08, -2.9802e-08,  0.0000e+00]])
tensor([[ 7.8357e-09, -3.2624e-08],
        [-1.9469e-08,  8.6988e-08],
        [ 3.8332e-08, -1.3337e-07],
        [ 3.1983e-08,  6.8157e-09],
        [-6.1283e-10, -4.6031e-08],
        [-1.3562e-07, -3.9085e-08]])


In [27]:
# Lemma 1
print(torch.linalg.svd(Sigma_tilde - W_hat @ W_hat.T).S)

tensor([3.7217, 3.6651, 3.5138, 1.9860, 1.5126, 1.0074])


In [34]:
# Lemma 2
print((Sigma_tilde @ Sigma_inv)) # Identity on block diagonal
print((Sigma_tilde - W_hat @ W_hat.T) @ Phi_inv)

tensor([[ 1.0000e+00,  1.7168e-08,  1.1194e-07,  6.4185e-03, -4.6447e-03,
         -4.1118e-03],
        [-5.0893e-08,  1.0000e+00, -5.1973e-08,  7.4838e-03, -5.4158e-03,
         -4.7945e-03],
        [ 1.7917e-07, -4.5378e-08,  1.0000e+00, -2.7382e-03,  1.9818e-03,
          1.7546e-03],
        [ 3.1021e-03,  6.1035e-03, -4.6053e-03,  1.0000e+00,  1.4080e-07,
          3.7851e-08],
        [-3.7459e-03, -7.3711e-03,  5.5631e-03,  2.8106e-07,  1.0000e+00,
         -7.2488e-08],
        [-2.1018e-03, -4.1363e-03,  3.1221e-03,  2.6460e-07,  2.1112e-07,
          1.0000e+00]])
tensor([[ 1.0000e+00, -8.5770e-08,  1.0978e-08,  6.4186e-03, -4.6447e-03,
         -4.1117e-03],
        [-1.1426e-08,  1.0000e+00, -7.3851e-09,  7.4838e-03, -5.4157e-03,
         -4.7945e-03],
        [ 9.6096e-08,  2.2991e-08,  1.0000e+00, -2.7381e-03,  1.9818e-03,
          1.7546e-03],
        [ 3.1023e-03,  6.1035e-03, -4.6052e-03,  1.0000e+00,  7.2222e-08,
         -1.3094e-08],
        [-3.7459e-03, -7.3710